### Data Reading

In [0]:
df = spark.read.format("parquet").load("abfss://bronze@databrickstestete.dfs.core.windows.net/orders")

In [0]:
df.display()

In [0]:
df.printSchema()

root
 |-- order_id: string (nullable = true)
 |-- customer_id: string (nullable = true)
 |-- product_id: string (nullable = true)
 |-- order_date: date (nullable = true)
 |-- quantity: integer (nullable = true)
 |-- total_amount: double (nullable = true)
 |-- _rescued_data: string (nullable = true)



In [0]:
df = df.withColumnRenamed("_rescued_data", "rescued_data")

In [0]:
df = df.drop("rescued_data")
df.display()

In [0]:
from pyspark.sql.functions import *

In [0]:
df =df.withColumn("order_date",to_timestamp(col("order_date")))

In [0]:
df = df.withColumn("year",year(col("order_date")))

In [0]:
from pyspark.sql.window import Window

In [0]:
df1 = df.withColumn("flag",dense_rank().over(Window.partitionBy("year").orderBy(desc("total_amount"))))

In [0]:
df1 = df1.withColumn("rank_flag",rank().over(Window.partitionBy("year").orderBy(desc("total_amount"))))
df1.display()

In [0]:
class WindowFunc:
    def __init__(self,df):
        self.df = df
    
    def dense_rank(self):
        df_dense_rank = self.df.withColumn("flag",dense_rank().over(Window.partitionBy("year").orderBy(desc("total_amount"))))
        return df_dense_rank
    
    def rank(self):
        df_rank = self.df.withColumn("rank_flag",rank().over(Window.partitionBy("year").orderBy(desc("total_amount"))))
        return df_rank
    
    def row_number(self):
        df_row_number = self.df.withColumn("row_number",row_number().over(Window.partitionBy("year").orderBy(desc("total_amount"))))
        return df_row_number

In [0]:
df_new=df

In [0]:
w = WindowFunc(df_new)
w.dense_rank().display()

In [0]:
w.rank().display()

In [0]:
df.write.format("delta").mode("overwrite").save("abfss://silver@databrickstestete.dfs.core.windows.net/orders")


In [0]:
%sql
CREATE TABLE IF NOT EXISTS databricks_cata.silver.order_silver
USING DELTA
LOCATION "abfss://silver@databrickstestete.dfs.core.windows.net/orders"
